# Publications markdown generator for academicpages

Takes a TSV of publications with metadata and converts them for use with [academicpages.github.io](academicpages.github.io). This is an interactive Jupyter notebook ([see more info here](http://jupyter-notebook-beginner-guide.readthedocs.io/en/latest/what_is_jupyter.html)). The core python code is also in `publications.py`. Run either from the `markdown_generator` folder after replacing `publications.tsv` with one containing your data.

TODO: Make this work with BibTex and other databases of citations, rather than Stuart's non-standard TSV format and citation style.


## Data format

The TSV needs to have the following columns: pub_date, title, venue, excerpt, citation, site_url, and paper_url, with a header at the top. 

- `excerpt` and `paper_url` can be blank, but the others must have values. 
- `pub_date` must be formatted as YYYY-MM-DD.
- `url_slug` will be the descriptive part of the .md file and the permalink URL for the page about the paper. The .md file will be `YYYY-MM-DD-[url_slug].md` and the permalink will be `https://[yourdomain]/publications/YYYY-MM-DD-[url_slug]`

This is how the raw file looks (it doesn't look pretty, use a spreadsheet or other program to edit and create).

In [56]:
!cat publications.tsv

pub_date	title	venue	excerpt	citation	url_slug	paper_url
2021-07-06	How Do Fractures Influence Hyporheic Exchange in Sedimentary Rock Riverbeds?	Water Resources Research	Data-Informed Hydrogeologic modelling.	R Chow, BL Parker, CM Steelman, A Thoms, W Nowak (2021). "How do fractures influence hyporheic exchange in sedimentary rock riverbeds?" <i>Water Resources Research</i>. 57(7).	j0_undergrad	http://adthoms.github.io/files/j0_undergad.pdf
2023-06-13	Tightly Coupled, Graph-Based DVL/IMU Fusion and Decoupled Mapping for SLAM-Centric Maritime Infrastructure Inspection	IEEE Journal of Oceanic Engineering	Ego-centric sensor-fusion and inspection tooling.	A Thoms, G Earle, N Charron, S Narasimhan (2023). "Tightly Coupled, Graph-Based DVL/IMU Fusion and Decoupled Mapping for SLAM-Centric Maritime Infrastructure Inspection" <i>IEEE Journal of Oceanic Engineering</i>. 48(3).	j0_phd	http://adthoms.github.io/files/j0_phd.pdf
2023-04-25	Graph-based structural joint pose estimation in non-line-of

## Import pandas

We are using the very handy pandas library for dataframes.

In [57]:
import pandas as pd

## Import TSV

Pandas makes this easy with the read_csv function. We are using a TSV, so we specify the separator as a tab, or `\t`.

I found it important to put this data in a tab-separated values format, because there are a lot of commas in this kind of data and comma-separated values can get messed up. However, you can modify the import statement, as pandas also has read_excel(), read_json(), and others.

In [58]:
publications = pd.read_csv("publications.tsv", sep="\t", header=0)
publications


,pub_date,title,venue,excerpt,citation,url_slug,paper_url
0,2021-07-06,How Do Fractures Influence Hyporheic Exchange ...,Water Resources Research,Data-Informed Hydrogeologic modelling.,"R Chow, BL Parker, CM Steelman, A Thoms, W Now...",j0_undergrad,http://adthoms.github.io/files/j0_undergad.pdf
1,2023-06-13,"Tightly Coupled, Graph-Based DVL/IMU Fusion an...",IEEE Journal of Oceanic Engineering,Ego-centric sensor-fusion and inspection tooling.,"A Thoms, G Earle, N Charron, S Narasimhan (202...",j0_phd,http://adthoms.github.io/files/j0_phd.pdf
2,2023-04-25,Graph-based structural joint pose estimation i...,Earthquake Engineering and Engineering Vibration,Exploring the application of SLAM to the post-...,"A Thoms, Z Al-Sabbag, S Narasimhan (2023). ""Gr...",j1_phd,http://adthoms.github.io/files/j1_phd.pdf


## Escape special characters

YAML is very picky about how it takes a valid string, so we are replacing single and double quotes (and ampersands) with their HTML encoded equivilents. This makes them look not so readable in raw format, but they are parsed and rendered nicely.

In [59]:
html_escape_table = {
    "&": "&amp;",
    '"': "&quot;",
    "'": "&apos;"
    }

def html_escape(text):
    """Produce entities within text."""
    return "".join(html_escape_table.get(c,c) for c in text)

## Creating the markdown files

This is where the heavy lifting is done. This loops through all the rows in the TSV dataframe, then starts to concatentate a big string (```md```) that contains the markdown for each type. It does the YAML metadata first, then does the description for the individual page.

In [60]:
import os
for row, item in publications.iterrows():
    
    md_filename = str(item.pub_date) + "-" + item.url_slug + ".md"
    html_filename = str(item.pub_date) + "-" + item.url_slug
    year = item.pub_date[:4]
    
    ## YAML variables
    
    md = "---\ntitle: \""   + item.title + '"\n'
    
    md += """collection: publications"""
    
    md += """\npermalink: /publication/""" + html_filename
    
    if len(str(item.excerpt)) > 5:
        md += "\nexcerpt: '" + html_escape(item.excerpt) + "'"
    
    md += "\ndate: " + str(item.pub_date) 
    
    md += "\nvenue: '" + html_escape(item.venue) + "'"
    
    if len(str(item.paper_url)) > 5:
        md += "\npaperurl: '" + item.paper_url + "'"
    
    md += "\ncitation: '" + html_escape(item.citation) + "'"
    
    md += "\n---"
    
    ## Markdown description for individual page
        
    if len(str(item.excerpt)) > 5:
        md += "\n" + html_escape(item.excerpt) + "\n"
    
    if len(str(item.paper_url)) > 5:
        md += "\n[Download paper here](" + item.paper_url + ")\n" 
        
    md += "\nRecommended citation: " + item.citation
    
    md_filename = os.path.basename(md_filename)
       
    with open("../_publications/" + md_filename, 'w') as f:
        f.write(md)

These files are in the publications directory, one directory below where we're working from.

In [61]:
!ls ../_publications/

2021-07-06-j0_undergrad.md  2023-04-25-j1_phd.md  2023-06-13-j0_phd.md
